### Imports

In [1]:
import torch
import shutil
import pickle

from simulation import citygraph_dataset
from learning import inductive_route_learning, eval_route_generator
from omegaconf import OmegaConf, DictConfig
from hydra import initialize_config_dir, compose
from simulation import drawing

from tqdm import tqdm
from pathlib import Path

### Graph dataset generating

In [2]:
dataset = citygraph_dataset.DynamicCityGraphDataset(
    min_nodes=50,
    max_nodes=50,
    edge_keep_prob=0.7,
    data_type=citygraph_dataset.MIXED,  # or any other type you want
    directed=False,
    fully_connected_demand=True,  # default SIDE_LENGTH_M
    mumford_style=True,
    pos_only=False
)

# Generate graphs
n_graphs = 10  # number of graphs you want to generate
graphs = [dataset.generate_graph(draw=False) for _ in tqdm(range(n_graphs))]

100%|██████████| 10/10 [00:00<00:00, 70.35it/s]


In [3]:
import pickle
from pathlib import Path

# Путь для сохранения
save_path = Path('./output_graphs')
if not save_path.exists():
    save_path.mkdir(parents=True)

# Сохраняем объект в файл
with open(save_path / 'raw_graphs_1000.pkl', 'wb') as ff:
    pickle.dump(graphs, ff)


In [4]:
import pickle

with open(save_path / 'raw_graphs_1000.pkl', 'rb') as f:
    graphs = pickle.load(f)

### Config load and learning construct

In [5]:
from omegaconf import OmegaConf
from hydra import initialize_config_dir, compose

with initialize_config_dir(config_dir="/root/TNDP_learning/cfg", job_name="app"):
    cfg = compose(config_name="ppo_20nodes_copy.yaml")  # загрузит config.yaml + всё из defaults
    # cfg = OmegaConf.to_container(cfg, resolve=False)  # опционально — привести к dict
    print(cfg)

/tmp/ipykernel_18095/3755569231.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir="/root/TNDP_learning/cfg", job_name="app"):


{'ppo': {'n_iterations': 200, 'val_period': 1, 'n_epochs': 1, 'minibatch_size': 2, 'horizon': 120, 'epsilon': 0.2, 'use_gae': True, 'gae_lambda': 0.95}, 'discount_rate': 0.95, 'diff_reward': True, 'baseline_lr': 0.0005, 'entropy_weight': 0.0, 'batch_size': 2, 'reward_scale': 1.0, 'lr': 0.0016134816080499328, 'decay': 0.0008404361781997002, 'optimizer': 'Adam', 'eval': {'n_routes': 10, 'min_route_len': 8, 'max_route_len': 10}, 'dataset': {'type': 'mumford', 'path': '/root/TNDP_learning/CEC2013Supp/Instances', 'city': 'VO'}, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.33, 'route_time_weight': 0.33, 'median_connectivity_weight': 0.33, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'model': {'common': {'dropout': 0.0, 'nonlin_type': 'ReLU', '

In [6]:
cfg['model']['backbone_gn']['kwargs']['in_edge_dim']

14

In [7]:
inductive_route_learning.setup_and_train(cfg)

  0%|          | 0/200 [00:00<?, ?it/s]2025-07-29 12:22:23.180 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 52 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component sizes:: [1, 57, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
2025-07-29 12:22:23.182 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:37 - Removing 51 nodes from 51 smaller strongly connected components. These are subgraphs where nodes are internally reachable but isolated from the rest. Retaining only the largest strongly connected component (57 nodes).
2025-07-29 12:22:24.185 | WARNING  | iduedu.utils.utils:keep_largest_strongly_connected_component:27 - Graph contains 54 weakly connected components. This means the graph has disconnected groups if edge directions are ignored. Component 

KeyboardInterrupt: 

### Evaluation 

In [ ]:
with initialize_config_dir(config_dir="/root/TNDP_learning/cfg", job_name="app"):
    cfg = compose(config_name="ppo_20nodes_copy.yaml",
                  overrides=[
                      "++model.weights=/root/TNDP_learning/output/inductive_gae_seed_1.pt",
                      "++eval.dataset.path=/root/TNDP_learning/CEC2013Supp/Instances",
                      "+eval=mandl",
                      "+run_name=my_mandl_lc100_2"
                  ])

In [ ]:
cfg

In [ ]:
DEVICE, run_name, _, cost_obj, model = eval_route_generator.lrnu.process_standard_experiment_cfg(
    cfg, 'nn_construction_', weights_required=True
)

test_ds = eval_route_generator.get_dataset_from_config(cfg.eval.dataset)
test_dl = eval_route_generator.DataLoader(test_ds, batch_size=cfg.batch_size)

n_samples = cfg.get('n_samples', None)
sbs = cfg.get('sample_batch_size', cfg.batch_size)
_, _, routes = eval_route_generator.eval_model(
    model, test_dl, cfg.eval, cost_obj,
    n_samples=n_samples, sample_batch_size=sbs, return_routes=True,
    device=DEVICE
)

eval_route_generator.dump_routes(run_name, routes.cpu())

In [ ]:
with open("/root/TNDP_learning/output_routes/nn_construction_my_mandl_lc100_2_routes.pkl", "rb") as f:
    routes = pickle.load(f)

In [ ]:
routes

In [ ]:
import torch

node_locs = []
with open("/root/TNDP_learning/CEC2013Supp/Instances/MandlCoords.txt") as f:
    lines = f.readlines()
    # Пропускаем первую строку (число узлов)
    for line in lines[1:]:
        parts = line.strip().split()
        if len(parts) == 2:
            x, y = float(parts[0]), float(parts[1])
            node_locs.append([x, y])
node_locs = torch.tensor(node_locs, dtype=torch.float)
print(node_locs.shape)  # Должно быть (15, 2)

In [ ]:
graph = torch.load("/root/TNDP_learning/output_graphs/mixed/processed/pre_transform.pt")

In [ ]:
type(graph)

In [ ]:
node_locs = graph[1]['stop']['pos']
print(node_locs.shape)

In [ ]:
print(type(routes))
print(len(routes))
print(routes[0].shape)
print(routes[0])

In [ ]:
routes_tensor = routes[0]  # если routes — список из одного элемента
routes_list = []
for row in routes_tensor:
    route = row[row != -1]  # убираем паддинги
    routes_list.append(route)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))

In [ ]:
!python 'learning/eval_route_generator.py' --help 

In [ ]:
config = {
    'experiment': {
        'logdir': None,
        'anomaly': False,
        'cpu': False,
        'seed': 0,
        'symmetric_routes': True,
        'cost_function': {
            'type': 'mine',
            'kwargs': {
                'mean_stop_time_s': 0,
                'avg_transfer_wait_time_s': 300,
                'demand_time_weight': 0.5,
                'route_time_weight': 0.5,
                'constraint_violation_weight': 5.0,
                'variable_weights': True,
                'pp_fraction': 0.33,
                'op_fraction': 0.33
            }
        }
    },
    'model': {
        'common': {
            'dropout': 0.0,
            'nonlin_type': 'ReLU',
            'embed_dim': 64
        },
        'route_generator': {
            'type': 'PathCombiningRouteGenerator',
            'kwargs': {
                'force_linking_unlinked': False,
                'logit_clip': None,
                'n_nodepair_layers': 3,
                'n_pathscorer_layers': 3,
                'pathscorer_hidden_dim': 16,
                'n_halt_layers': 3,
                'halt_scorer_type': 'endpoints',
                'serial_halting': True
            }
        },
        'backbone_gn': {
            'net_type': 'graph attn',
            'kwargs': {
                'n_layers': 5,
                'in_node_dim': 4,
                'in_edge_dim': 13,
                'use_norm': False,
                'n_heads': 4,
                'dense': False
            }
        },
        'weights': '/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output/inductive_30_06_2025_18_12_45.pt'
    },
    'n_samples': 100,
    'batch_size': 512,
    'eval': {
        'csv': True,
        'n_routes': 6,
        'min_route_len': 2,
        'max_route_len': 8,
        'dataset': {
            'type': 'mumford',
            'path': '/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/CEC2013Supp-2/Instances',
            'city': 'Mandl'
        }
    },
    'run_name': 'my_mandl_lc100_2'
}

### Бэнчмарк граф 

In [ ]:
!python 'learning/eval_route_generator.py' +model.weights=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output/inductive_30_06_2025_18_12_45.pt \
    eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/mumford_dataset/Instances \
    +eval=mandl +run_name=mandl_lc100_test

In [ ]:
!python -m scripts.data_display.plot_generated_networks cfg/eval/mandl.yaml \
    --routes output_routes/neural_bco_14_07_2025_17:58:20_routes.pkl \
    -o plots/mandl_routes_test_neural_bco.png

In [ ]:
!python learning/bee_colony.py eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/mumford_dataset/Instances \
    +eval=mandl \
    init.path=output_routes/nn_construction_mandl_lc100_test_routes.pkl

In [ ]:
!pip install IduEdu

In [ ]:
!python learning/bee_colony.py -h

In [ ]:
!python learning/bee_colony.py --config-name neural_bco_mumford \
    +model.weights=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output/inductive_30_06_2025_18_12_45.pt \
    eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/mumford_dataset/Instances \
    +eval=mandl \
    init.path=output_routes/nn_construction_mandl_lc100_test_routes.pkl \
    >> bco_test_pareto.csv

### Vaska

In [ ]:
!python 'learning/eval_route_generator.py' +model.weights=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output/inductive_30_06_2025_18_12_45.pt \
    eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/vo_dataset \
    +eval=VO +run_name=vo_test \
    n_samples=100 


In [ ]:
!python learning/bee_colony.py eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/vo_dataset \
    +eval=VO \
    init.path=output_routes/nn_construction_vo_test_routes.pkl

In [ ]:
!python learning/bee_colony.py --config-name neural_bco_mumford \
    +model.weights=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output/inductive_30_06_2025_18_12_45.pt \
    eval.dataset.path=/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/datasets/vo_dataset \
    +eval=VO \
    init.path=output_routes/nn_construction_vo_test_routes.pkl 

In [ ]:
!python 'learning/eval_route_generator.py' -h

In [ ]:
!python -m scripts.data_display.plot_generated_networks --help

In [ ]:
!python -m scripts.data_display.plot_generated_networks cfg/eval/mandl.yaml \
    --routes output_routes/.pkl \
    -o plots/mandl_routes.png

In [ ]:
!python -m scripts.data_display.plot_generated_networks cfg/eval/vo.yaml \
    --routes output_routes/neural_bco_15_07_2025_16:10:08_routes.pkl \
    -o plots/vo_routes_test_bco_neural.png

In [ ]:
!python -m scripts.data_display.plot_pareto \
    VO_pareto.csv \
    -o pareto.png